Step 1: Broze Layer - To update the json records from github to the delta lake to table.

In [0]:
# Simple setup - download CVE data and prepare bronze layer
import os
import json
import zipfile
import urllib.request
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

print(" Setting up CVE data for Graph Analytics demo...")
print(" Target: 2000 CVEs from 2024 (Bronze)")

# Temporary working directory for downloads/extractions
TMP_DIR = "/tmp/cve_demo"
os.makedirs(TMP_DIR, exist_ok=True)
print(f"📁 Temporary workspace: {TMP_DIR}")


🚀 Setting up CVE data for Graph Analytics demo...
📊 Target: 2000 CVEs from 2024 (Bronze)
📁 Temporary workspace: /tmp/cve_demo


In [0]:
print("📥 Downloading CVE repository for 2024 Bronze load...")

# Temp locations on the driver (local to the workspace runtime)
zip_dest    = "/tmp/cve_graph_demo/cvelistV5.zip"
extract_dir = "/tmp/cve_graph_demo/cvelistV5-main"

# Create working directory
os.makedirs("/tmp/cve_graph_demo", exist_ok=True)

# Download the repository ZIP (full CVE list V5)
zip_url = "https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip"
print(f"📥 Downloading from: {zip_url}")

with urllib.request.urlopen(zip_url) as response:
    data = response.read()

with open(zip_dest, "wb") as f:
    f.write(data)

print(f"✅ Downloaded {len(data):,} bytes to {zip_dest}")

# Extract the ZIP so we can traverse cves/2024/*
print("📦 Extracting ZIP archive...")
with zipfile.ZipFile(zip_dest) as z:
    z.extractall("/tmp/cve_graph_demo/")

print(f"✅ Extraction complete at: {extract_dir}")
print("✅ Bronze 2024 Delta target (managed table): bronze_db.cve_bronze_2024")


📥 Downloading CVE repository for 2024 Bronze load...
📥 Downloading from: https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip
✅ Downloaded 524,600,837 bytes to /tmp/cve_graph_demo/cvelistV5.zip
📦 Extracting ZIP archive...
✅ Extraction complete at: /tmp/cve_graph_demo/cvelistV5-main
✅ Bronze 2024 Delta target (managed table): bronze_db.cve_bronze_2024


In [0]:
# Process 2024 CVEs (no limit if max_files=None)
print("🎯 Processing 2024 CVEs...")

def process_year_cves(year, max_files=None):
    """Process CVEs for a specific year. If max_files is None, process all files."""
    cve_year_dir = f"{extract_dir}/cves/{year}"
    json_files = []
    print(f"📂 Scanning directory: {cve_year_dir}")

    if os.path.exists(cve_year_dir):
        file_count = 0
        for root, dirs, files in os.walk(cve_year_dir):  # recurse through subfolders
            # stop the outer loop too if we've hit the cap
            if max_files is not None and file_count >= max_files:
                break

            for file in files:
                if file.endswith(".json") and f"CVE-{year}-" in file:
                    if max_files is not None and file_count >= max_files:
                        break

                    file_path = os.path.join(root, file)
                    try:
                        with open(file_path, "r", encoding="utf-8") as f:
                            cve_data = json.load(f)   # load directly from file
                            json_files.append(cve_data)

                        file_count += 1
                        if file_count % 500 == 0:
                            print(f"   📊 Processed {file_count} CVE-{year} files...")

                    except Exception as e:
                        print(f"⚠️ Skipped {file}: {e}")
                        continue

        print(f"✅ Collected {len(json_files)} CVEs from {year}")
        return json_files
    else:
        print(f"❌ Directory not found: {cve_year_dir}")
        return []

# ALL 2024 files (no cap)
cves_2024 = process_year_cves(2024)   # pass a number to cap if needed later
print(f"📊 Total 2024 CVEs collected: {len(cves_2024)}")


🎯 Processing 2024 CVEs...
📂 Scanning directory: /tmp/cve_graph_demo/cvelistV5-main/cves/2024
   📊 Processed 500 CVE-2024 files...
   📊 Processed 1000 CVE-2024 files...
   📊 Processed 1500 CVE-2024 files...
   📊 Processed 2000 CVE-2024 files...
   📊 Processed 2500 CVE-2024 files...
   📊 Processed 3000 CVE-2024 files...
   📊 Processed 3500 CVE-2024 files...
   📊 Processed 4000 CVE-2024 files...
   📊 Processed 4500 CVE-2024 files...
   📊 Processed 5000 CVE-2024 files...
   📊 Processed 5500 CVE-2024 files...
   📊 Processed 6000 CVE-2024 files...
   📊 Processed 6500 CVE-2024 files...
   📊 Processed 7000 CVE-2024 files...
   📊 Processed 7500 CVE-2024 files...
   📊 Processed 8000 CVE-2024 files...
   📊 Processed 8500 CVE-2024 files...
   📊 Processed 9000 CVE-2024 files...
   📊 Processed 9500 CVE-2024 files...
   📊 Processed 10000 CVE-2024 files...
   📊 Processed 10500 CVE-2024 files...
   📊 Processed 11000 CVE-2024 files...
   📊 Processed 11500 CVE-2024 files...
   📊 Processed 12000 CVE-2024 

In [0]:
from pyspark.sql.functions import col


In [0]:
# --- Serverless-safe Bronze writer for ALL 2024 CVEs (no pandas, no sparkContext) ---

from pyspark.sql.functions import (
    current_timestamp,
    current_date,
    lit,
    monotonically_increasing_id,
    col,
)
import json

print("💾 Converting 2024 CVEs to Spark DataFrame and saving as a managed Delta table...")
print("🚀 No pandas, no sparkContext, no Arrow config changes — serverless-friendly.")

# 1) Make sure the DB exists (managed tables => no filesystem paths needed)
spark.sql("CREATE DATABASE IF NOT EXISTS bronze_db")

table_name = "bronze_db.cve_bronze_2024"

def save_cves_to_delta_managed(cves_list, year: int, table_name: str):
    """
    Save CVEs (list[dict]) to a MANAGED Delta table.

    Works on serverless because:
      * no pandas
      * no sparkContext
      * no spark.conf tweaks
    """
    if not cves_list:
        print(f"⚠️ No CVEs to save for {year}")
        return None

    print(f"📊 Normalizing and converting {len(cves_list)} CVEs to Spark DataFrame...")

    # --- KEY FIX: normalize the 'containers' field so Spark can infer a single type ---
    normalized = []
    for rec in cves_list:
        r = dict(rec)  # shallow copy so we don't mutate original list

        cont = r.get("containers", None)
        if cont is None:
            # keep it explicitly as None
            r["containers"] = None
        else:
            # always store as JSON string -> consistent StringType for all rows
            try:
                r["containers"] = json.dumps(cont)
            except Exception:
                # if something is really weird, just stringify it
                r["containers"] = json.dumps(str(cont))

        normalized.append(r)

    # list[dict] -> Spark DF (Spark infers nested schema for other fields; 'containers' is now string)
    df_raw = spark.createDataFrame(normalized)

    # Bronze metadata columns (same as before)
    df_bronze = (
        df_raw
        .withColumn("_ingestion_timestamp", current_timestamp())
        .withColumn("_ingestion_date", current_date())
        .withColumn("_year", lit(year))
        .withColumn("_record_id", monotonically_increasing_id())
    )

    record_count = df_bronze.count()
    print(f"📈 {year} CVE records going into table: {record_count:,}")

    # Write as MANAGED Delta table in the metastore
    (
        df_bronze.write
        .format("delta")
        .mode("overwrite")          # replace if you re-run the notebook
        .option("mergeSchema", "true")
        .option("overwriteSchema", "true")
        .option("delta.columnMapping.mode", "name")
        .saveAsTable(table_name)
    )

    print(f"✅ {year} Bronze managed table created/updated: {table_name}")
    return df_bronze

# 2) Save ALL 2024 CVEs to the managed Delta table
df_2024 = save_cves_to_delta_managed(cves_2024, 2024, table_name)

# 3) Verify the table
print("🔍 Verifying managed table read...")
bronze_2024 = spark.table(table_name)
print(f"✅ 2024 data loaded: {bronze_2024.count():,} records\n")

print("🔎 Sample 2024 (key fields):")
(
    bronze_2024
    .select(
        col("cveMetadata.cveId").alias("cve_id"),
        col("cveMetadata.datePublished").alias("date_published"),
    )
    .show(10, truncate=False)
)

print("\n📄 Table storage details:")
spark.sql(f"DESCRIBE DETAIL {table_name}").show(truncate=False)

print("\n🕘 Delta history:")
spark.sql(f"DESCRIBE HISTORY {table_name}").show(truncate=False)


💾 Converting 2024 CVEs to Spark DataFrame and saving as a managed Delta table...
🚀 No pandas, no sparkContext, no Arrow config changes — serverless-friendly.
📊 Normalizing and converting 38753 CVEs to Spark DataFrame...
📈 2024 CVE records going into table: 38,753
✅ 2024 Bronze managed table created/updated: bronze_db.cve_bronze_2024
🔍 Verifying managed table read...
✅ 2024 data loaded: 38,753 records

🔎 Sample 2024 (key fields):
+--------------+------------------------+
|cve_id        |date_published          |
+--------------+------------------------+
|CVE-2024-42629|2024-08-12T00:00:00     |
|CVE-2024-42391|2024-11-18T09:07:03.674Z|
|CVE-2024-42636|2024-08-23T00:00:00     |
|CVE-2024-42143|2024-07-30T07:46:36.904Z|
|CVE-2024-42831|2024-10-07T00:00:00     |
|CVE-2024-42308|2024-08-17T09:09:13.467Z|
|CVE-2024-42136|2024-07-30T07:46:30.975Z|
|CVE-2024-42163|2024-08-12T11:21:54.443Z|
|CVE-2024-42103|2024-07-30T07:45:59.220Z|
|CVE-2024-42052|2024-07-28T00:00:00.000Z|
+--------------+-----

In [0]:
# Verify the data is ready for graph analytics (2024 Bronze only, managed table)
print("🔍 Verifying data for graph analytics...")

from pyspark.sql.functions import col  # <-- add this import

try:
    table_name = "bronze_db.cve_bronze_2024"
    bronze_2024 = spark.table(table_name)

    # Count
    cnt = bronze_2024.count()
    print(f"✅ 2024 data loaded: {cnt} records")

    # Sample key fields
    print("\n🔍 Sample 2024 data (key fields for graph):")
    bronze_2024.select(
        col("cveMetadata.cveId").alias("cve_id"),
        col("cveMetadata.datePublished").alias("date_published")
    ).show(5, truncate=False)

    print("\n🚀 SUCCESS! Data is ready for Graph Analytics demo!")
    print(f"📚 Bronze table: {table_name}")

    # (Optional) storage details
    spark.sql(f"DESCRIBE DETAIL {table_name}").show(truncate=False)

except Exception as e:
    print(f"❌ Error verifying data: {e}")
    print("Check that the table bronze_db.cve_bronze_2024 exists, then re-run.")


🔍 Verifying data for graph analytics...
✅ 2024 data loaded: 38753 records

🔍 Sample 2024 data (key fields for graph):
+--------------+------------------------+
|cve_id        |date_published          |
+--------------+------------------------+
|CVE-2024-42629|2024-08-12T00:00:00     |
|CVE-2024-42391|2024-11-18T09:07:03.674Z|
|CVE-2024-42636|2024-08-23T00:00:00     |
|CVE-2024-42143|2024-07-30T07:46:36.904Z|
|CVE-2024-42831|2024-10-07T00:00:00     |
+--------------+------------------------+
only showing top 5 rows

🚀 SUCCESS! Data is ready for Graph Analytics demo!
📚 Bronze table: bronze_db.cve_bronze_2024
+------+------------------------------------+-----------------------------------+-----------+--------+-----------------------+-------------------+----------------+-----------------+--------+-----------+---------------------------------------------------------------------------------------------------------------+----------------+----------------+--------------------------------------

In [0]:
%sql
-- Sanity check of the table created
DESCRIBE DETAIL bronze_db.cve_bronze_2024;

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,fe30b537-6264-40d4-9f8e-41f263e1d618,workspace.bronze_db.cve_bronze_2024,null,,2025-11-12T20:47:08.393Z,2025-11-16T17:58:29.000Z,List(),List(),8,38262743,"Map(delta.columnMapping.mode -> name, delta.enableDeletionVectors -> true, delta.columnMapping.maxColumnId -> 269)",3,7,"List(appendOnly, columnMapping, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
%sql
-- count of the table created
SELECT count(*) FROM bronze_db.cve_bronze_2024 ;

count(*)
38753


In [0]:
%sql
select * from bronze_db.cve_bronze_2024
limit 5

containers,cveMetadata,dataType,dataVersion,_ingestion_timestamp,_ingestion_date,_year,_record_id
"{""cna"": {""providerMetadata"": {""orgId"": ""8254265b-2729-46b6-b9e3-3dfca2d5bfca"", ""shortName"": ""mitre"", ""dateUpdated"": ""2024-08-12T15:47:36.184640""}, ""descriptions"": [{""lang"": ""en"", ""value"": ""FrogCMS v0.9.5 was discovered to contain a Cross-Site Request Forgery (CSRF) vulnerability via /admin/?/page/edit/10.""}], ""affected"": [{""vendor"": ""n/a"", ""product"": ""n/a"", ""versions"": [{""version"": ""n/a"", ""status"": ""affected""}]}], ""references"": [{""url"": ""https://github.com/Kirtoc/cms/tree/main/2/readme.md""}], ""problemTypes"": [{""descriptions"": [{""type"": ""text"", ""lang"": ""en"", ""description"": ""n/a""}]}]}, ""adp"": [{""problemTypes"": [{""descriptions"": [{""type"": ""CWE"", ""cweId"": ""CWE-352"", ""lang"": ""en"", ""description"": ""CWE-352 Cross-Site Request Forgery (CSRF)""}]}], ""affected"": [{""vendor"": ""frogcms_project"", ""product"": ""frogcms"", ""cpes"": [""cpe:2.3:a:frogcms_project:frogcms:0.9.5:*:*:*:*:*:*:*""], ""defaultStatus"": ""unknown"", ""versions"": [{""version"": ""0.9.5"", ""status"": ""affected""}]}], ""metrics"": [{""cvssV3_1"": {""scope"": ""CHANGED"", ""version"": ""3.1"", ""baseScore"": 5.4, ""attackVector"": ""NETWORK"", ""baseSeverity"": ""MEDIUM"", ""vectorString"": ""CVSS:3.1/AV:N/AC:L/PR:L/UI:R/S:C/C:L/I:L/A:N"", ""integrityImpact"": ""LOW"", ""userInteraction"": ""REQUIRED"", ""attackComplexity"": ""LOW"", ""availabilityImpact"": ""NONE"", ""privilegesRequired"": ""LOW"", ""confidentialityImpact"": ""LOW""}}, {""other"": {""type"": ""ssvc"", ""content"": {""timestamp"": ""2024-08-12T17:10:52.488455Z"", ""id"": ""CVE-2024-42629"", ""options"": [{""Exploitation"": ""poc""}, {""Automatable"": ""no""}, {""Technical Impact"": ""partial""}], ""role"": ""CISA Coordinator"", ""version"": ""2.0.3""}}}], ""title"": ""CISA ADP Vulnrichment"", ""providerMetadata"": {""orgId"": ""134c704f-9b21-4f2e-91b3-4a467353bcc0"", ""shortName"": ""CISA-ADP"", ""dateUpdated"": ""2024-08-12T17:11:48.084Z""}}]}","Map(datePublished -> 2024-08-12T00:00:00, cveId -> CVE-2024-42629, assignerOrgId -> 8254265b-2729-46b6-b9e3-3dfca2d5bfca, state -> PUBLISHED, assignerShortName -> mitre, dateReserved -> 2024-08-05T00:00:00, dateUpdated -> 2024-08-12T17:11:48.084Z)",CVE_RECORD,5.1,2025-11-16T17:58:26.776Z,2025-11-16,2024,14532
"{""cna"": {""affected"": [{""collectionURL"": ""https://github.com/cesanta/mongoose"", ""defaultStatus"": ""unaffected"", ""product"": ""Mongoose Web Server"", ""vendor"": ""Cesanta"", ""versions"": [{""lessThanOrEqual"": ""7.14"", ""status"": ""affected"", ""version"": ""0"", ""versionType"": ""semver""}]}], ""credits"": [{""lang"": ""en"", ""type"": ""finder"", ""value"": ""Gabriele Quagliarella""}], ""descriptions"": [{""lang"": ""en"", ""supportingMedia"": [{""base64"": false, ""type"": ""text/html"", ""value"": ""Use of Out-of-range Pointer Offset vulnerability in Cesanta Mongoose Web Server v7.14 allows an attacker to send an unexpected TLS packet and force the application to read unintended heap memory space.""}], ""value"": ""Use of Out-of-range Pointer Offset vulnerability in Cesanta Mongoose Web Server v7.14 allows an attacker to send an unexpected TLS packet and force the application to read unintended heap memory space.""}], ""metrics"": [{""cvssV3_1"": {""attackComplexity"": ""LOW"", ""attackVector"": ""NETWORK"", ""availabilityImpact"": ""NONE"", ""baseScore"": 4.3, ""baseSeverity"": ""MEDIUM"", ""confidentialityImpact"": ""LOW"", ""integrityImpact"": ""NONE"", ""privilegesRequired"": ""NONE"", ""scope"": ""UNCHANGED"", ""userInteraction"": ""REQUIRED"", ""vectorString"": ""CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:U/C:L/I:N/A:N"", ""version"": ""3.1""}, ""format"": ""CVSS"", ""scenarios"": [{""lang"": ""en"", ""value"": ""GENERAL""}]}], ""problemTypes"": [{""descriptions"": [{""cweId"": ""CWE-823"", ""description"": ""CWE-823 Use of Out-of-range Pointer Offset